In [1]:
%load_ext watermark
%watermark

Last updated: 2024-06-09T17:34:35.043063-05:00

Python implementation: CPython
Python version       : 3.10.11
IPython version      : 8.13.2

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing, feature_extraction
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
#import category_encoders
from sklearn.model_selection import cross_validate
from category_encoders import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.compose import ColumnTransformer

In [3]:
consumidores = pd.read_csv("C:/Users/Danny/Desktop/Analisis tesis/4 - Pipelines/data/df_consumo_varObjetivo.csv")

In [4]:
consumidores.head()

,departamento,residentes_hogar,d2_04_num_hijos,per_edad_tipo,estrato_tipo,situacion_tipo,frecuencia_consumo_marihuana_tipo,frecuencia_consumo_cocaina_tipo,frecuencia_consumo_basuco_tipo,per_sexo_tipo,...,g_06_a_posibilidad_conseguir_marihuana_tipo,g_06_b_posibilidad_conseguir_cocaina_tipo,g_06_c_posibilidad_conseguir_basuco_tipo,g_07_alguien_ofrecio_comprar_probar_sp_tipo,g_01_a_num_familiares_consumen_sp_imp_tipo,g_02_a_num_amigos_consumen_sp_imp_tipo,g_08_a_ofrecieron_marihuana_imp_tipo,g_08_b_ofrecieron_cocaina_imp_tipo,g_08_c_ofrecieron_basuco_imp_tipo,CatRiesgo
0,Guaviare,5,0,Joven,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
1,Cordoba,4,3,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
2,Caqueta,4,2,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
3,Bolivar,6,0,Adulto,Estrato alto,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_No_podria_conseguir,g_06_b_No_podria_conseguir,g_06_c_No_podria_conseguir,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
4,Tolima,4,1,Joven,Estrato bajo,Buscando trabajo,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,1


Como vimos en un apartado anterior, en este modelo vamos a modificar cada variable en función de su tipo. Al conjunto de pasos que siguen los datos se le llama comúnmente **Pipelines** (literalmente, sistemas de tuberias).

In [5]:
consumidores.shape

(10000, 40)

**INCISO**

vamos a modificar dos transformadores de scikitlearn para que sean compatibles con pipelines. Este paso es necesario en la version actual de scikit-learn, pero seguramente será arreglado en el futuro

In [6]:
variable_objetivo = ["CatRiesgo"]
variables_independientes = consumidores.drop(variable_objetivo, axis=1).columns

In [7]:
datos_numericos = consumidores[variables_independientes].select_dtypes([int, float])
col_numericas = datos_numericos.columns
col_no_numericas = consumidores[variables_independientes].select_dtypes([object]).columns

dict_var_ordinales = {
    "per_edad_tipo": ["Adolescente", "Joven", "Adulto", "Persona mayor"],
    "estrato_tipo":["Estrato bajo", "Estrato medio", "Estrato alto"],
    "d_08_estado_salud_tipo":["d_08_Muy_malo","d_08_Malo","d_08_Regular","d_08_Bueno","d_08_Muy_bueno"],
    "frecuencia_consumo_marihuana_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_cocaina_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_basuco_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
}

col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = consumidores[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = consumidores[col_categoricas]

Un pipeline de sklearn se define como una secuencia de pasos. Cada paso se define con una tupla de forma `[nombre del paso, transformador]`

Por ejemplo, si queremos crear un pipeline que procese las variables numéricas, primero imputándolas y después estandarizandolas, podriamos crear un pipeline

Creamos ahora los pipelines para cada tipo de variable. Es necesario aclarar que se va ha hacer uso de la libreria Category Encoders par ael procesamiento de variables categoricas y categoricas ordinales

In [8]:
transformador_numerico = Pipeline([
    ("imputador_numerico", SimpleImputer(missing_values=np.nan, strategy='mean')),
    ("escalador_numerico", preprocessing.MinMaxScaler())
])

pipeline_numerico = ColumnTransformer([
    ("transformador_numerico", transformador_numerico, col_numericas),
])

pipeline_numerico.fit_transform(consumidores)[:5]

array([[0.30769231, 0.        ],
       [0.23076923, 0.25      ],
       [0.23076923, 0.16666667],
       [0.38461538, 0.        ],
       [0.23076923, 0.08333333]])

In [9]:
pipeline_numerico

ColumnTransformer(transformers=[('transformador_numerico',
                                 Pipeline(steps=[('imputador_numerico',
                                                  SimpleImputer()),
                                                 ('escalador_numerico',
                                                  MinMaxScaler())]),
                                 Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])

Ahora se procede a crear el pipeline categorico

In [10]:
transformador_categorico = ColumnTransformer([
    ("transformador_categorico", OneHotEncoder(handle_unknown="impute"), col_categoricas)
])

pipeline_categorico = Pipeline([
    ("transformador_categorico", transformador_categorico)
])

pipeline_categorico.fit_transform(consumidores)[:5]

array([[1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
pipeline_categorico

Pipeline(steps=[('transformador_categorico',
                 ColumnTransformer(transformers=[('transformador_categorico',
                                                  OneHotEncoder(handle_unknown='impute'),
                                                  ['d_09_deprimido_tipo',
                                                   'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                   'departamento',
                                                   'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                   'd2_01_etnia_tipo',
                                                   'g_08_c_ofrecieron_basuco_imp_tipo',
                                                   'g_02_amigos_consu...
                                                   'd_10_poco_interes_tipo',
                                                   'd_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo',
                                                   'per_sexo_tipo',
                                                   'd2_05_nivel_educativo_tipo',
                                                   'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                   'd_12_b_presenta_problema_consumo_sp_barrio_tipo',
                                                   'g_01_familiares_consumen_sp_tipo',
                                                   'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                   'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                   'vivienda_tipo', ...])]))])

`category_encoders.OrdinalEncoders` requiere un mapeado específico para cada columna, indicando el número por el que queremos reemplazar cada nivel de cada variable ordinal. Si no especificamos un mapa, asume que el orden es aleatorio.

In [12]:
niveles_ordinales = []
for col, levels in dict_var_ordinales.items():
    niveles_ordinales.append({"col": col, 
                              "mapping": dict(zip(levels, range(len(levels))))})

In [13]:
transformador_ordinal = ColumnTransformer([
    ("transformador_ordinal", OrdinalEncoder(mapping=niveles_ordinales), col_ordinales)
])

pipeline_ordinal = Pipeline([
    ("transformador_ordinal", transformador_ordinal)
])
pipeline_ordinal.fit_transform(consumidores)[:5]

array([[1, 0, 1, 0, 0, 0],
       [2, 0, 2, 0, 0, 0],
       [2, 0, 3, 0, 0, 0],
       [2, 2, 3, 0, 0, 0],
       [1, 0, 3, 0, 0, 0]])

Ya tenemos una manera de, dado un conjunto de datos, separarlos y aplicar distintas transformaciones a cada variable. Nos falta una manera de, una vez se han transformado, reunirlas de nuevo.

Para ello podemos usar `FeatureUnion`, que simplemente toma un conjunto de pasos de un pipeline y los une.

In [14]:
pipeline_procesado = FeatureUnion([
    ('variables_numericas', pipeline_numerico),
    ('variables_ordinales', pipeline_ordinal),
    ('variables_categoricas', pipeline_categorico),
])

In [15]:
pipeline_procesado

FeatureUnion(transformer_list=[('variables_numericas',
                                ColumnTransformer(transformers=[('transformador_numerico',
                                                                 Pipeline(steps=[('imputador_numerico',
                                                                                  SimpleImputer()),
                                                                                 ('escalador_numerico',
                                                                                  MinMaxScaler())]),
                                                                 Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                               ('variables_ordinales',
                                Pipeline(steps=[('transformador_ordinal',
                                                 ColumnTransfor...
                                                                                   'd_10_poco_interes_tipo',
                                                                                   'd_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo',
                                                                                   'per_sexo_tipo',
                                                                                   'd2_05_nivel_educativo_tipo',
                                                                                   'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                   'd_12_b_presenta_problema_consumo_sp_barrio_tipo',
                                                                                   'g_01_familiares_consumen_sp_tipo',
                                                                                   'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                   'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                   'vivienda_tipo', ...])]))]))])

In [16]:
consumidores_procesado = pipeline_procesado.fit_transform(consumidores)
consumidores_procesado.shape

(10000, 145)

Finalmente, necesitamos añadir un estimador al final para predecir en base a los datos transformados, pero primero se separa los datos en datos de entrenamiento y datos de test

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
     consumidores[variables_independientes], consumidores["CatRiesgo"],
     test_size=0.20, random_state=42
)

In [18]:
estimador = LogisticRegression(C=2.3118996566114403, max_iter=13000, penalty='l1', solver='saga')

Se debe realizar la separacion de datos en datos de entrenamiento y datos de test y ademas se debe establecer el estimador a usar en el modelo

In [19]:
pipeline_estimador = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', estimador)
])

Para el fit se pasan los datos que se obtienen de entrenamiento y para el predict los datos de test

In [20]:
pipeline_estimador.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'd2_05_nivel_educativo_tipo',
                                                                                                    'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                                    'd_12_b_presenta_problema_consumo_sp_barrio_tipo',
                                                                                                    'g_01_familiares_consumen_sp_tipo',
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'vivienda_tipo', ...])]))]))])),
                ('estimador',
                 LogisticRegression(C=2.3118996566114403, max_iter=13000,
                                    penalty='l1', solver='saga'))])

In [21]:
predicciones = pipeline_estimador.predict(X_test)

In [22]:
predicciones

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

Se evalua el modelo segun lo obtenido

El beneficio de los pipelines, no solo es tener codigo mas legible y poder gestionar de forma ordenada todo el ciclo de vida del modelado, sino que los pipelines tienen todos los beneficios de los objetos de scikitlearn, por ejemplo, podemos usar validacion cruzada directamente con el pipeline.

In [23]:
resultados = {}

def exactitud_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.accuracy_score(y, preds)

def precision_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.precision_score(y, preds, average="weighted")

def exhaustividad_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.recall_score(y, preds, average="weighted")

def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.f1_score(y, preds, average="weighted")

In [24]:
def evaluar_modelo(prmPipeline,X, y, semillas):
    restultados_estimador = {}
    restultados_estimador["exactitud"] = cross_validate(prmPipeline, X, y,
                                           scoring=exactitud_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["precision"] = cross_validate(prmPipeline, X, y,
                                           scoring=precision_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["exhaustividad"] = cross_validate(prmPipeline, X, y,
                                           scoring=exhaustividad_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["f1_score"] = cross_validate(prmPipeline, X, y,
                                           scoring=f1_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    return restultados_estimador

In [25]:
#Funcion para poder visuallizar los resultados de las metricas
def ver_resultados():
    df_res = pd.DataFrame()
    for modelo, metricas in resultados.items():
        for metrica, valores in metricas.items():
            for medida, arreglo in valores.items():
                # Calcula la media de cada array y agrega una nueva fila al DataFrame
                df_res.loc[f"{modelo}_{metrica}", medida] = np.mean(arreglo)
                
    return df_res

In [26]:
estimador_agregado = pipeline_estimador.named_steps['estimador']
resultados["rl"] = evaluar_modelo(estimador_agregado, consumidores_procesado, consumidores.CatRiesgo, 50)

In [27]:
ver_resultados()

,fit_time,score_time,test_score,train_score
rl_exactitud,241.304866,0.001932,0.986500,0.997690
rl_precision,235.055344,0.004661,0.987063,0.997689
rl_exhaustividad,229.659379,0.004780,0.986500,0.997690
rl_f1_score,225.416735,0.005681,0.986478,0.997689


### Exportar con joblib el mejor pipeline obtenido

Tenemos que exportar 3 cosas para poder recrear el Pipeline:

**1. El pipeline entrenado.**

Podemos usar la libreria de scikit-learn `joblib` para guardar nuestro pipeline al disco duro. Joblib es una libreria basada en pickle que sirve para guardar objetos de python al disco duro, pero que está optimizado para scikit-learn.

In [28]:
import joblib

In [70]:
joblib.dump(pipeline_estimador_op_best4, 'pipeline_rl_best.pkl')

['pipeline_rl_best.pkl']

In [29]:
clf = joblib.load('C:/Users/Danny/Desktop/Analisis_Modelo/Alisis Danny/4 - Pipelines/Modelo RL/pipeline_rl_best.pkl')

In [30]:
array_predicciones = clf.predict(X_test)

In [31]:
array_predicciones

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [34]:
import json
with open('columnas_consumidores.json', 'w') as fname:
    consumidores_columnas = consumidores.columns.tolist()
    json.dump(consumidores_columnas, fname)

In [35]:
consumidores_dtypes = consumidores.dtypes
consumidores_dtypes = {col: consumidores[col].dtype for col in consumidores.columns}
joblib.dump(consumidores_dtypes, 'dtypes_consumidores.pkl')

['dtypes_consumidores.pkl']

In [36]:
obs = {
  "d2_01_etnia_tipo": "d2_01_Ninguno",
  "d2_03_estado_civil_tipo": "d2_03_Soltero",
  "d2_04_num_hijos": 0,
  "d2_05_nivel_educativo_tipo": "d2_05_Media",
  "d_01_aporta_dinero_hogar_tipo": "d_01_Si",
  "d_08_estado_salud_tipo": "d_08_Malo",
  "d_09_deprimido_tipo": "d_09_Si",
  "d_10_poco_interes_tipo": "d_10_Si",
  "d_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo": "d_11_h_Gran_riesgo",
  "d_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo": "d_11_k_Gran_riesgo",
  "d_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo": "d_11_n_Gran_riesgo",
  "d_12_b_presenta_problema_consumo_sp_barrio_tipo": "d_12_b_No",
  "d_12_c_presenta_problema_expendio_sp_barrio_tipo": "d_12_c_No",
  "departamento": "Guaviare",
  "estrato_tipo": "Estrato bajo",
  "frecuencia_consumo_basuco_tipo": "Nunca",
  "frecuencia_consumo_cocaina_tipo": "Nunca",
  "frecuencia_consumo_marihuana_tipo": "Nunca",
  "g_01_a_num_familiares_consumen_sp_imp_tipo": "g_01_a_No_aplica",
  "g_01_familiares_consumen_sp_tipo": "g_01_No",
  "g_02_a_num_amigos_consumen_sp_imp_tipo": "g_02_a_Dos_o_mas",
  "g_02_amigos_consumen_sp_tipo": "g_02_Si",
  "g_03_curiosidad_probar_sp_tipo": "g_03_No",
  "g_04_probaria_sp_tipo": "g_04_No",
  "g_05_posibilidad_probar_sp_tipo": "g_05_No",
  "g_06_a_posibilidad_conseguir_marihuana_tipo": "g_06_a_No_conoce_facilidad_dificultad",
  "g_06_b_posibilidad_conseguir_cocaina_tipo": "g_06_b_No_conoce_facilidad_dificultad",
  "g_06_c_posibilidad_conseguir_basuco_tipo": "g_06_c_No_conoce_facilidad_dificultad",
  "g_07_alguien_ofrecio_comprar_probar_sp_tipo": "g_07_No",
  "g_08_a_ofrecieron_marihuana_imp_tipo": "g_08_a_No_aplica",
  "g_08_b_ofrecieron_cocaina_imp_tipo": "g_08_b_No_aplica",
  "g_08_c_ofrecieron_basuco_imp_tipo": "g_08_c_No_aplica",
  "per_edad_tipo": "Joven",
  "per_sexo_tipo": "Hombre",
  "residentes_hogar": 5,
  "situacion_tipo": "Trabajando",
  "vive_madre_hogar_tipo": "vive_madre_hogar_No",
  "vive_padre_hogar_tipo": "vive_padre_hogar_No",
  "vivienda_tipo": "Apartamento"
}

In [37]:
def dict_a_df(obs, columnas, dtypes):
    obs_df = pd.DataFrame([obs])
    for col, dtype in dtypes.items():
        if col in obs_df.columns:
            obs_df[col] = obs_df[col].astype(dtype)
        else:
            obs_df[col] = None 
    return obs_df

obs_df = dict_a_df(obs, consumidores_columnas, consumidores_dtypes)
obs_df

,d2_01_etnia_tipo,d2_03_estado_civil_tipo,d2_04_num_hijos,d2_05_nivel_educativo_tipo,d_01_aporta_dinero_hogar_tipo,d_08_estado_salud_tipo,d_09_deprimido_tipo,d_10_poco_interes_tipo,d_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo,d_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo,...,g_08_b_ofrecieron_cocaina_imp_tipo,g_08_c_ofrecieron_basuco_imp_tipo,per_edad_tipo,per_sexo_tipo,residentes_hogar,situacion_tipo,vive_madre_hogar_tipo,vive_padre_hogar_tipo,vivienda_tipo,CatRiesgo
0,d2_01_Ninguno,d2_03_Soltero,0,d2_05_Media,d_01_Si,d_08_Malo,d_09_Si,d_10_Si,d_11_h_Gran_riesgo,d_11_k_Gran_riesgo,...,g_08_b_No_aplica,g_08_c_No_aplica,Joven,Hombre,5,Trabajando,vive_madre_hogar_No,vive_padre_hogar_No,Apartamento,None


In [38]:
import shap

In [39]:
procesador = clf.named_steps['procesador']
estimador_back = clf.named_steps['estimador']

In [64]:
obs_df = obs_df.reindex(columns=consumidores.columns)

In [65]:
obs_df

,departamento,residentes_hogar,d2_04_num_hijos,per_edad_tipo,estrato_tipo,situacion_tipo,frecuencia_consumo_marihuana_tipo,frecuencia_consumo_cocaina_tipo,frecuencia_consumo_basuco_tipo,per_sexo_tipo,...,g_06_a_posibilidad_conseguir_marihuana_tipo,g_06_b_posibilidad_conseguir_cocaina_tipo,g_06_c_posibilidad_conseguir_basuco_tipo,g_07_alguien_ofrecio_comprar_probar_sp_tipo,g_01_a_num_familiares_consumen_sp_imp_tipo,g_02_a_num_amigos_consumen_sp_imp_tipo,g_08_a_ofrecieron_marihuana_imp_tipo,g_08_b_ofrecieron_cocaina_imp_tipo,g_08_c_ofrecieron_basuco_imp_tipo,CatRiesgo
0,Guaviare,5,0,Joven,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,None


In [66]:
nueva_data_transformada = procesador.transform(obs_df)

In [67]:
prediccion_nueva = estimador_back.predict(nueva_data_transformada)
prediccion_nueva

array([0], dtype=int64)

In [72]:
#!pip install lime

In [73]:
import lime
import lime.lime_tabular

In [85]:
consumidores_columnas

['departamento',
 'residentes_hogar',
 'd2_04_num_hijos',
 'per_edad_tipo',
 'estrato_tipo',
 'situacion_tipo',
 'frecuencia_consumo_marihuana_tipo',
 'frecuencia_consumo_cocaina_tipo',
 'frecuencia_consumo_basuco_tipo',
 'per_sexo_tipo',
 'vive_padre_hogar_tipo',
 'vive_madre_hogar_tipo',
 'vivienda_tipo',
 'd_01_aporta_dinero_hogar_tipo',
 'd_08_estado_salud_tipo',
 'd_09_deprimido_tipo',
 'd_10_poco_interes_tipo',
 'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
 'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
 'd_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo',
 'd_12_b_presenta_problema_consumo_sp_barrio_tipo',
 'd_12_c_presenta_problema_expendio_sp_barrio_tipo',
 'd2_01_etnia_tipo',
 'd2_03_estado_civil_tipo',
 'd2_05_nivel_educativo_tipo',
 'g_01_familiares_consumen_sp_tipo',
 'g_02_amigos_consumen_sp_tipo',
 'g_03_curiosidad_probar_sp_tipo',
 'g_04_probaria_sp_tipo',
 'g_05_posibilidad_probar_sp_tipo',
 'g_06_a_posibilidad_conseguir_marihu

In [86]:
array_col_proces = ['transformador_numerico__residentes_hogar',
 'transformador_numerico__d2_04_num_hijos',
 'transformador_ordinal__per_edad_tipo',
 'transformador_ordinal__estrato_tipo',
 'transformador_ordinal__d_08_estado_salud_tipo',
 'transformador_ordinal__frecuencia_consumo_marihuana_tipo',
 'transformador_ordinal__frecuencia_consumo_cocaina_tipo',
 'transformador_ordinal__frecuencia_consumo_basuco_tipo',
 'transformador_categorico__d2_05_nivel_educativo_tipo_1',
 'transformador_categorico__d2_05_nivel_educativo_tipo_2',
 'transformador_categorico__d2_05_nivel_educativo_tipo_3',
 'transformador_categorico__d2_05_nivel_educativo_tipo_4',
 'transformador_categorico__d2_05_nivel_educativo_tipo_5',
 'transformador_categorico__d2_05_nivel_educativo_tipo_6',
 'transformador_categorico__d2_05_nivel_educativo_tipo_7',
 'transformador_categorico__d2_05_nivel_educativo_tipo_8',
 'transformador_categorico__g_07_alguien_ofrecio_comprar_probar_sp_tipo_1',
 'transformador_categorico__g_07_alguien_ofrecio_comprar_probar_sp_tipo_2',
 'transformador_categorico__g_08_a_ofrecieron_marihuana_imp_tipo_1',
 'transformador_categorico__g_08_a_ofrecieron_marihuana_imp_tipo_2',
 'transformador_categorico__g_08_a_ofrecieron_marihuana_imp_tipo_3',
 'transformador_categorico__g_08_a_ofrecieron_marihuana_imp_tipo_4',
 'transformador_categorico__g_08_a_ofrecieron_marihuana_imp_tipo_5',
 'transformador_categorico__d_01_aporta_dinero_hogar_tipo_1',
 'transformador_categorico__d_01_aporta_dinero_hogar_tipo_2',
 'transformador_categorico__d_10_poco_interes_tipo_1',
 'transformador_categorico__d_10_poco_interes_tipo_2',
 'transformador_categorico__g_01_familiares_consumen_sp_tipo_1',
 'transformador_categorico__g_01_familiares_consumen_sp_tipo_2',
 'transformador_categorico__departamento_1',
 'transformador_categorico__departamento_2',
 'transformador_categorico__departamento_3',
 'transformador_categorico__departamento_4',
 'transformador_categorico__departamento_5',
 'transformador_categorico__departamento_6',
 'transformador_categorico__departamento_7',
 'transformador_categorico__departamento_8',
 'transformador_categorico__departamento_9',
 'transformador_categorico__departamento_10',
 'transformador_categorico__departamento_11',
 'transformador_categorico__departamento_12',
 'transformador_categorico__departamento_13',
 'transformador_categorico__departamento_14',
 'transformador_categorico__departamento_15',
 'transformador_categorico__departamento_16',
 'transformador_categorico__departamento_17',
 'transformador_categorico__departamento_18',
 'transformador_categorico__departamento_19',
 'transformador_categorico__departamento_20',
 'transformador_categorico__departamento_21',
 'transformador_categorico__departamento_22',
 'transformador_categorico__departamento_23',
 'transformador_categorico__departamento_24',
 'transformador_categorico__departamento_25',
 'transformador_categorico__departamento_26',
 'transformador_categorico__departamento_27',
 'transformador_categorico__departamento_28',
 'transformador_categorico__departamento_29',
 'transformador_categorico__departamento_30',
 'transformador_categorico__departamento_31',
 'transformador_categorico__departamento_32',
 'transformador_categorico__g_02_amigos_consumen_sp_tipo_1',
 'transformador_categorico__g_02_amigos_consumen_sp_tipo_2',
 'transformador_categorico__g_06_b_posibilidad_conseguir_cocaina_tipo_1',
 'transformador_categorico__g_06_b_posibilidad_conseguir_cocaina_tipo_2',
 'transformador_categorico__g_06_b_posibilidad_conseguir_cocaina_tipo_3',
 'transformador_categorico__g_06_b_posibilidad_conseguir_cocaina_tipo_4',
 'transformador_categorico__d_09_deprimido_tipo_1',
 'transformador_categorico__d_09_deprimido_tipo_2',
 'transformador_categorico__g_05_posibilidad_probar_sp_tipo_1',
 'transformador_categorico__g_05_posibilidad_probar_sp_tipo_2',
 'transformador_categorico__d_12_b_presenta_problema_consumo_sp_barrio_tipo_1',
 'transformador_categorico__d_12_b_presenta_problema_consumo_sp_barrio_tipo_2',
 'transformador_categorico__g_04_probaria_sp_tipo_1',
 'transformador_categorico__g_04_probaria_sp_tipo_2',
 'transformador_categorico__g_06_a_posibilidad_conseguir_marihuana_tipo_1',
 'transformador_categorico__g_06_a_posibilidad_conseguir_marihuana_tipo_2',
 'transformador_categorico__g_06_a_posibilidad_conseguir_marihuana_tipo_3',
 'transformador_categorico__g_06_a_posibilidad_conseguir_marihuana_tipo_4',
 'transformador_categorico__g_02_a_num_amigos_consumen_sp_imp_tipo_1',
 'transformador_categorico__g_02_a_num_amigos_consumen_sp_imp_tipo_2',
 'transformador_categorico__g_02_a_num_amigos_consumen_sp_imp_tipo_3',
 'transformador_categorico__d_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo_1',
 'transformador_categorico__d_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo_2',
 'transformador_categorico__d_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo_3',
 'transformador_categorico__d_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo_4',
 'transformador_categorico__d_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo_5',
 'transformador_categorico__situacion_tipo_1',
 'transformador_categorico__situacion_tipo_2',
 'transformador_categorico__situacion_tipo_3',
 'transformador_categorico__situacion_tipo_4',
 'transformador_categorico__d2_01_etnia_tipo_1',
 'transformador_categorico__d2_01_etnia_tipo_2',
 'transformador_categorico__d2_01_etnia_tipo_3',
 'transformador_categorico__d2_01_etnia_tipo_4',
 'transformador_categorico__d2_01_etnia_tipo_5',
 'transformador_categorico__d2_01_etnia_tipo_6',
 'transformador_categorico__g_08_c_ofrecieron_basuco_imp_tipo_1',
 'transformador_categorico__g_08_c_ofrecieron_basuco_imp_tipo_2',
 'transformador_categorico__g_08_c_ofrecieron_basuco_imp_tipo_3',
 'transformador_categorico__g_08_c_ofrecieron_basuco_imp_tipo_4',
 'transformador_categorico__g_08_c_ofrecieron_basuco_imp_tipo_5',
 'transformador_categorico__d_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo_1',
 'transformador_categorico__d_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo_2',
 'transformador_categorico__d_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo_3',
 'transformador_categorico__d_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo_4',
 'transformador_categorico__d_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo_5',
 'transformador_categorico__d_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo_1',
 'transformador_categorico__d_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo_2',
 'transformador_categorico__d_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo_3',
 'transformador_categorico__d_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo_4',
 'transformador_categorico__d_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo_5',
 'transformador_categorico__g_01_a_num_familiares_consumen_sp_imp_tipo_1',
 'transformador_categorico__g_01_a_num_familiares_consumen_sp_imp_tipo_2',
 'transformador_categorico__g_01_a_num_familiares_consumen_sp_imp_tipo_3',
 'transformador_categorico__per_sexo_tipo_1',
 'transformador_categorico__per_sexo_tipo_2',
 'transformador_categorico__vivienda_tipo_1',
 'transformador_categorico__vivienda_tipo_2',
 'transformador_categorico__vivienda_tipo_3',
 'transformador_categorico__vivienda_tipo_4',
 'transformador_categorico__d2_03_estado_civil_tipo_1',
 'transformador_categorico__d2_03_estado_civil_tipo_2',
 'transformador_categorico__d2_03_estado_civil_tipo_3',
 'transformador_categorico__d2_03_estado_civil_tipo_4',
 'transformador_categorico__d2_03_estado_civil_tipo_5',
 'transformador_categorico__g_03_curiosidad_probar_sp_tipo_1',
 'transformador_categorico__g_03_curiosidad_probar_sp_tipo_2',
 'transformador_categorico__d_12_c_presenta_problema_expendio_sp_barrio_tipo_1',
 'transformador_categorico__d_12_c_presenta_problema_expendio_sp_barrio_tipo_2',
 'transformador_categorico__g_06_c_posibilidad_conseguir_basuco_tipo_1',
 'transformador_categorico__g_06_c_posibilidad_conseguir_basuco_tipo_2',
 'transformador_categorico__g_06_c_posibilidad_conseguir_basuco_tipo_3',
 'transformador_categorico__g_06_c_posibilidad_conseguir_basuco_tipo_4',
 'transformador_categorico__vive_padre_hogar_tipo_1',
 'transformador_categorico__vive_padre_hogar_tipo_2',
 'transformador_categorico__vive_padre_hogar_tipo_3',
 'transformador_categorico__vive_madre_hogar_tipo_1',
 'transformador_categorico__vive_madre_hogar_tipo_2',
 'transformador_categorico__vive_madre_hogar_tipo_3',
 'transformador_categorico__g_08_b_ofrecieron_cocaina_imp_tipo_1',
 'transformador_categorico__g_08_b_ofrecieron_cocaina_imp_tipo_2',
 'transformador_categorico__g_08_b_ofrecieron_cocaina_imp_tipo_3',
 'transformador_categorico__g_08_b_ofrecieron_cocaina_imp_tipo_4',
 'transformador_categorico__g_08_b_ofrecieron_cocaina_imp_tipo_5']

In [87]:
array_col_proces

['transformador_numerico__residentes_hogar',
 'transformador_numerico__d2_04_num_hijos',
 'transformador_ordinal__per_edad_tipo',
 'transformador_ordinal__estrato_tipo',
 'transformador_ordinal__d_08_estado_salud_tipo',
 'transformador_ordinal__frecuencia_consumo_marihuana_tipo',
 'transformador_ordinal__frecuencia_consumo_cocaina_tipo',
 'transformador_ordinal__frecuencia_consumo_basuco_tipo',
 'transformador_categorico__d2_05_nivel_educativo_tipo_1',
 'transformador_categorico__d2_05_nivel_educativo_tipo_2',
 'transformador_categorico__d2_05_nivel_educativo_tipo_3',
 'transformador_categorico__d2_05_nivel_educativo_tipo_4',
 'transformador_categorico__d2_05_nivel_educativo_tipo_5',
 'transformador_categorico__d2_05_nivel_educativo_tipo_6',
 'transformador_categorico__d2_05_nivel_educativo_tipo_7',
 'transformador_categorico__d2_05_nivel_educativo_tipo_8',
 'transformador_categorico__g_07_alguien_ofrecio_comprar_probar_sp_tipo_1',
 'transformador_categorico__g_07_alguien_ofrecio_compr

In [102]:
num_features = len(array_col_proces)
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=procesador.transform(obs_df),
    feature_names=array_col_proces,
    class_names=[0, 1, 2, 3, 4],
    mode='classification'
)

In [103]:
i = 0# Índice de la observación que quieres explicar
num_features = len(array_col_proces)
exp = explainer.explain_instance(
    data_row=nueva_data_transformada[i],
    predict_fn=estimador_back.predict_proba,
    num_features=num_features
)

In [104]:
lista_importancia_caracteristicas = exp.as_list()

In [106]:
len(importancia_caracteristicas)

145

In [109]:
importancia_caracteristicas

[('transformador_categorico__d_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo_4 <= 0.00',
  0.23142418275684654),
 ('transformador_ordinal__frecuencia_consumo_marihuana_tipo <= 0.00',
  0.21097429748271648),
 ('transformador_categorico__d_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo_5 <= 0.00',
  0.1736632957034706),
 ('transformador_categorico__d2_01_etnia_tipo_6 <= 0.00',
  -0.13388827881544993),
 ('transformador_categorico__d_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo_5 <= 0.00',
  -0.1319967646627495),
 ('transformador_categorico__departamento_31 <= 0.00', 0.1190317475653376),
 ('transformador_categorico__d2_05_nivel_educativo_tipo_8 <= 0.00',
  0.1024607268873534),
 ('transformador_categorico__departamento_30 <= 0.00', -0.10103522846028955),
 ('transformador_categorico__g_07_alguien_ofrecio_comprar_probar_sp_tipo_2 <= 0.00',
  -0.09662178474177471),
 ('transformador_categorico__departamento_2 <= 0.00', 0.08171739358956952),
 ('transformador_catego

In [112]:
lista_caracteristicas = []
lista_val_caracteristicas = []
for feature, importance in importancia_caracteristicas:
    #print(f"Característica: {feature}, Importancia: {importance}")
    partes = feature.split(' ')
    if partes[0].startswith('transformador'):
        lista_caracteristicas.append(partes[0])
    elif partes[1].startswith('transformador'):
        lista_caracteristicas.append(partes[1])
    elif partes[2].startswith('transformador'):
        lista_caracteristicas.append(partes[2])
    lista_val_caracteristicas.append(importance)

In [114]:
len(lista_caracteristicas)

145

In [116]:
len(lista_val_caracteristicas)

145

In [119]:
resultados = []

# Iterar sobre cada elemento del arreglo original
for elemento in lista_caracteristicas:
    # Dividir la cadena por el caracter '_' y seleccionar la última parte
    partes = elemento.split('__')
    ultimo_parte = partes[-1]
                
    # Agregar la última parte al nuevo arreglo
    resultados.append(ultimo_parte)

In [120]:
importancia_caracteristicas_dict_0 = dict(zip(resultados, lista_val_caracteristicas))

In [121]:
importancia_caracteristicas_dict_0

{'d_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo_4': 0.23142418275684654,
 'frecuencia_consumo_marihuana_tipo': 0.21097429748271648,
 'd_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo_5': 0.1736632957034706,
 'd2_01_etnia_tipo_6': -0.13388827881544993,
 'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo_5': -0.1319967646627495,
 'departamento_31': 0.1190317475653376,
 'd2_05_nivel_educativo_tipo_8': 0.1024607268873534,
 'departamento_30': -0.10103522846028955,
 'g_07_alguien_ofrecio_comprar_probar_sp_tipo_2': -0.09662178474177471,
 'departamento_2': 0.08171739358956952,
 'departamento_15': 0.07833334317811722,
 'departamento_13': 0.07623333270815523,
 'g_03_curiosidad_probar_sp_tipo_1': -0.07606406931772619,
 'g_05_posibilidad_probar_sp_tipo_2': -0.07467202067452754,
 'g_08_a_ofrecieron_marihuana_imp_tipo_2': -0.07146160607844176,
 'g_02_amigos_consumen_sp_tipo_2': 0.06751021771915207,
 'g_08_a_ofrecieron_marihuana_imp_tipo_3': -0.06595466162093636,
 'depar

In [122]:
def obtenerDictProcesado(pipeline_columnas,importancia_caracteristicas_dict):
    nuevo_dict_caracteristicas_procesadas = {}
    sumatoria = 0
    promedio = 0
    num_caracteristicas = 0;
    for columna in pipeline_columnas:
        
        if columna == "d2_04_num_hijos":
            nuevo_dict_caracteristicas_procesadas["d2_04_num_hijos"] = importancia_caracteristicas_dict["d2_04_num_hijos"]
        else:
            for clave in importancia_caracteristicas_dict:
                if columna in clave:
                    sumatoria = sumatoria + importancia_caracteristicas_dict[clave]
                    num_caracteristicas = num_caracteristicas + 1
            
            if num_caracteristicas > 1:
                promedio = sumatoria / num_caracteristicas
                nuevo_dict_caracteristicas_procesadas[columna] = promedio
            else:
                nuevo_dict_caracteristicas_procesadas[columna] = sumatoria
            
            sumatoria = 0
            promedio = 0
            num_caracteristicas = 0;
    
    return nuevo_dict_caracteristicas_procesadas

In [123]:
dict_procesado_0 = obtenerDictProcesado(consumidores_columnas, importancia_caracteristicas_dict_0)

In [124]:
dict_procesado_0

{'departamento': 0.00797340532293455,
 'residentes_hogar': -0.009511624001329912,
 'd2_04_num_hijos': -0.008422974290515334,
 'per_edad_tipo': 0.024748298492117123,
 'estrato_tipo': -0.022788469044514554,
 'situacion_tipo': 0.016214187896635594,
 'frecuencia_consumo_marihuana_tipo': 0.21097429748271648,
 'frecuencia_consumo_cocaina_tipo': -0.023600622789504602,
 'frecuencia_consumo_basuco_tipo': 0.006552896414674883,
 'per_sexo_tipo': 0.007059168077096756,
 'vive_padre_hogar_tipo': 0.004895554746548826,
 'vive_madre_hogar_tipo': -0.0034375732269138373,
 'vivienda_tipo': -0.0008851828326290055,
 'd_01_aporta_dinero_hogar_tipo': 0.00024572886699109476,
 'd_08_estado_salud_tipo': -0.033078521124818266,
 'd_09_deprimido_tipo': -0.009620686682137728,
 'd_10_poco_interes_tipo': -0.01092654728518964,
 'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo': 0.012337821931543886,
 'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo': -0.03882632511698051,
 'd_11_n_conocimiento_rie